# Capstone Project: Comment Subtopics Analysis for Airbnb Hosts
---

How can a host on Airbnb understand that are their strengths and weaknesses? How can hosts point out the demand trend of their customers from a large scale of comments? This project focuses on using machine learning tools to help hosts understand the underlying trends of the comments on their property.  

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Capstone-Project:-Comment-Subtopics-Analysis-for-Airbnb-Hosts" data-toc-modified-id="Capstone-Project:-Comment-Subtopics-Analysis-for-Airbnb-Hosts-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Capstone Project: Comment Subtopics Analysis for Airbnb Hosts</a></span></li><li><span><a href="#Part-3.1:-LDA-Analysis-On-Comments-Based-On-Sentiment" data-toc-modified-id="Part-3.1:-LDA-Analysis-On-Comments-Based-On-Sentiment-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part 3.1: LDA Analysis On Comments Based On Sentiment</a></span><ul class="toc-item"><li><span><a href="#Import-Review-Data" data-toc-modified-id="Import-Review-Data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import Review Data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Import-Review-Data-Steps" data-toc-modified-id="Import-Review-Data-Steps-2.1.0.1"><span class="toc-item-num">2.1.0.1&nbsp;&nbsp;</span>Import Review Data Steps</a></span></li></ul></li></ul></li><li><span><a href="#LDA-Prep" data-toc-modified-id="LDA-Prep-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LDA Prep</a></span><ul class="toc-item"><li><span><a href="#Step1:-CountVectorize" data-toc-modified-id="Step1:-CountVectorize-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Step1: CountVectorize</a></span></li><li><span><a href="#Positive-Sentiment-Dictionary" data-toc-modified-id="Positive-Sentiment-Dictionary-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>Positive Sentiment Dictionary</a></span></li><li><span><a href="#LDA-Positive-Sentiment" data-toc-modified-id="LDA-Positive-Sentiment-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>LDA Positive Sentiment</a></span></li><li><span><a href="#Negative-Sentiment-Dictionary" data-toc-modified-id="Negative-Sentiment-Dictionary-2.2.4"><span class="toc-item-num">2.2.4&nbsp;&nbsp;</span>Negative Sentiment Dictionary</a></span></li></ul></li><li><span><a href="#LDA-Run" data-toc-modified-id="LDA-Run-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LDA Run</a></span><ul class="toc-item"><li><span><a href="#LDA-Negative-Sentiment" data-toc-modified-id="LDA-Negative-Sentiment-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>LDA Negative Sentiment</a></span></li></ul></li></ul></li></ul></div>


# Part 3.1: LDA Analysis On Comments Based On Sentiment
---

In [3]:
# !pip install --upgrade pyldavis
# !pip install --upgrade gensim

In [2]:
# !pip install langdetect

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

from gensim import corpora, models
import pyLDAvis.gensim
from langdetect import detect

pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

np.random.seed(42)

## Import Review Data 
---

#### Import Review Data Steps 
1. import data
2. select 2 columns: reivews, and sentiment 
3. break dataframe into 2, one with positive sentiment, the other with negative 
4. run LDA on both 

In [2]:
review_sentiments = pd.read_csv("../data/reviews_sentiment_score.csv", index_col = 0)

In [3]:
review_sentiments.head()

,key_0,listing_id,id,date,reviewer_id,reviewer_name,comments,language,overall_rating,compound,neg,neu,pos
0,0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st...",en,97.0,0.9590,0.000,0.788,0.212
1,1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...,en,97.0,0.9819,0.000,0.697,0.303
2,2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...,en,97.0,0.7600,0.134,0.710,0.156
3,3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...,en,97.0,0.9840,0.035,0.646,0.319
4,4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...,en,97.0,0.9617,0.000,0.613,0.387


In [4]:
review_sentiments.dtypes

key_0               int64
listing_id          int64
id                  int64
date               object
reviewer_id         int64
reviewer_name      object
comments           object
language           object
overall_rating    float64
compound          float64
neg               float64
neu               float64
pos               float64
dtype: object

In [5]:
positive_df = review_sentiments[review_sentiments['pos'] > 0.5]

In [6]:
positive_df.shape

(38588, 13)

In [7]:
negative_df = review_sentiments[review_sentiments['compound'] < -0.3]

In [8]:
negative_df.shape

(1610, 13)

## LDA Prep
---

### Step1: CountVectorize 

### Positive Sentiment Dictionary 
---

In [9]:
#for positive sentiment 
cv = CountVectorizer(ngram_range= (2,2), 
                     stop_words= 'english', 
                     min_df = 2)
cv_positive = cv.fit_transform(positive_df['comments'])

In [10]:
cv_positive_df =  pd.DataFrame(cv_positive.toarray(), columns = cv.get_feature_names())

In [11]:
cv_positive_df.head()

,00 night,00 pm,10 00,10 00pm,10 10,10 11pm,10 12,10 15,10 15min,10 20,...,zoo close,zoo enjoyed,zoo golden,zoo great,zoo ocean,zoo right,zoo went,über rides,정말 좋았습니다,좋았습니다 호스트
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
positive_texts = [cv_positive_df.columns[cv_positive_df.loc[index,:].nonzero()] for index in cv_positive_df.index]

In [14]:
dictionary_pos = corpora.Dictionary(positive_texts)

In [15]:
dictionary_pos

In [16]:
corpus_pos = [dictionary_pos.doc2bow(text) for text in positive_texts]

### LDA Positive Sentiment

In [17]:
ldamodel_pos = models.ldamodel.LdaModel(corpus_pos,                     # pass in our corpus
                                    id2word = dictionary_pos,       # matches each word to its "number" or "spot" in the dictionary
                                    num_topics = 15,             # number of topics T to find
                                    passes = 5,                 # number of passes through corpus; similar to number of epochs
                                    minimum_probability = 0.01)

In [18]:
pyLDAvis.gensim.prepare(ldamodel_pos, corpus_pos, dictionary_pos)

/Users/evelyn/anaconda3/envs/dsi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.259196 -0.023167       1        1  16.279964
2      0.184868  0.087507       2        1   9.753704
13     0.115090 -0.218853       3        1   8.141093
9      0.075045 -0.013687       4        1   7.906845
10     0.084520 -0.009512       5        1   7.520496
7      0.061969  0.146298       6        1   7.310721
12     0.020546  0.205714       7        1   7.106418
0      0.009311  0.017933       8        1   6.688818
3     -0.022192 -0.147544       9        1   5.418060
1     -0.087851 -0.054892      10        1   5.137886
14    -0.083032 -0.041258      11        1   5.096248
11    -0.151829  0.017647      12        1   4.509614
4     -0.138968 -0.002871      13        1   3.370803
5     -0.150669  0.023736      14        1   3.282056
6     -0.176007  0.012949      15        1   2.477268, topic_info=      Category         Freq                   Term        Total  loglift  \
term                                                                       
8      Default  5659.000000         great location  5659.000000  30.0000   
25     Default  3472.000000            great place  3472.000000  29.0000   
200    Default  3840.000000             place stay  3840.000000  28.0000   
176    Default  7805.000000          san francisco  7805.000000  27.0000   
208    Default  2416.000000            golden gate  2416.000000  26.0000   
207    Default  1994.000000              gate park  1994.000000  25.0000   
149    Default  1853.000000           enjoyed stay  1853.000000  24.0000   
19     Default  2362.000000            place great  2362.000000  23.0000   
246    Default  1374.000000         really enjoyed  1374.000000  22.0000   
37     Default  4445.000000       highly recommend  4445.000000  21.0000   
2117   Default  1077.000000          good location  1077.000000  20.0000   
1710   Default  1001.000000             nice place  1001.000000  19.0000   
219    Default  2853.000000       walking distance  2853.000000  18.0000   
2993   Default  3208.000000             great host  3208.000000  17.0000   
174    Default  1099.000000          quick respond  1099.000000  16.0000   
178    Default  1214.000000               stay san  1214.000000  15.0000   
699    Default  1079.000000            really nice  1079.000000  14.0000   
1109   Default  1578.000000  public transportation  1578.000000  13.0000   
2252   Default   735.000000       great experience   735.000000  12.0000   
330    Default  1395.000000            easy access  1395.000000  11.0000   
665    Default  1069.000000            super clean  1069.000000  10.0000   
5      Default  2877.000000        definitely stay  2877.000000   9.0000   
1932   Default   656.000000           coffee shops   656.000000   8.0000   
1162   Default   546.000000      exactly described   546.000000   7.0000   
2586   Default   731.000000           union square   731.000000   6.0000   
572    Default  1387.000000             room clean  1387.000000   5.0000   
639    Default  1266.000000      recommend staying  1266.000000   4.0000   
4327   Default   477.000000          overall great   477.000000   3.0000   
9781   Default   428.000000    great communication   428.000000   2.0000   
697    Default   488.000000            clean place   488.000000   1.0000   
...        ...          ...                    ...          ...      ...   
395    Topic15   280.225098           days arrival   281.665710   3.6929   
12626  Topic15   146.808655            close beach   148.249268   3.6882   
319    Topic15   117.817444       really recommend   119.258064   3.6859   
19605  Topic15    99.661713        bathroom shower   101.102333   3.6837   
6697   Topic15    98.684990     location wonderful   100.125610   3.6835   
1902   Topic15    98.111504             host house    99.552124   3.6834   
3014   Topic15    97.005798               try stay    98.446419   3.6833   

### Negative Sentiment Dictionary 
---

In [19]:
#for positive sentiment 
cv = CountVectorizer(ngram_range= (2,2), 
                     stop_words= 'english', 
                     min_df = 2)
cv_negative = cv.fit_transform(negative_df['comments'])

In [20]:
cv_negative_df =  pd.DataFrame(cv_negative.toarray(), columns = cv.get_feature_names())

In [21]:
negative_texts = [cv_negative_df.columns[cv_negative_df.loc[index,:].nonzero()] for index in cv_negative_df.index]

In [22]:
dictionary_neg = corpora.Dictionary(negative_texts)

In [23]:
corpus_neg = [dictionary_neg.doc2bow(text) for text in negative_texts]

## LDA Run
---

### LDA Negative Sentiment 

In [24]:
ldamodel_neg = models.ldamodel.LdaModel(corpus_neg, 
                                        id2word = dictionary_neg, 
                                        num_topics = 12, 
                                        passes = 5, 
                                        minimum_probability = 0.01)

In [25]:
pyLDAvis.gensim.prepare(ldamodel_neg, corpus_neg, dictionary_neg)

/Users/evelyn/anaconda3/envs/dsi/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.047009  0.025738       1        1  10.261383
4      0.015182 -0.086520       2        1  10.039733
0      0.085715  0.010215       3        1   9.794269
6      0.061782 -0.011999       4        1   9.602648
3      0.073271 -0.019708       5        1   8.898481
10     0.007716 -0.112852       6        1   8.524595
7     -0.045066 -0.004307       7        1   8.300357
1      0.007441  0.054025       8        1   8.000946
8     -0.148095 -0.056779       9        1   7.356337
2      0.010395  0.109011      10        1   6.997625
9     -0.028920  0.003466      11        1   6.608325
5     -0.086428  0.089712      12        1   5.615297, topic_info=     Category        Freq                  Term       Total  loglift  logprob
term                                                                         
68    Default   87.000000            place stay   87.000000  30.0000  30.0000
79    Default   50.000000            great stay   50.000000  29.0000  29.0000
371   Default   26.000000         good location   26.000000  28.0000  28.0000
218   Default   54.000000        location great   54.000000  27.0000  27.0000
35    Default  131.000000        great location  131.000000  26.0000  26.0000
277   Default   74.000000           great place   74.000000  25.0000  25.0000
265   Default   78.000000            great host   78.000000  24.0000  24.0000
270   Default   27.000000              stay san   27.000000  23.0000  23.0000
784   Default   22.000000            nice clean   22.000000  22.0000  22.0000
249   Default   45.000000           place great   45.000000  21.0000  21.0000
390   Default  186.000000         san francisco  186.000000  20.0000  20.0000
688   Default   23.000000      location perfect   23.000000  19.0000  19.0000
39    Default   78.000000      walking distance   78.000000  18.0000  18.0000
95    Default   69.000000       definitely stay   69.000000  17.0000  17.0000
138   Default   52.000000           golden gate   52.000000  16.0000  16.0000
137   Default   44.000000             gate park   44.000000  15.0000  15.0000
1727  Default    8.000000     arrival automated    8.000000  14.0000  14.0000
1728  Default    8.000000     automated posting    8.000000  13.0000  13.0000
986   Default   15.000000      friendly helpful   15.000000  12.0000  12.0000
1183  Default   24.000000            great time   24.000000  11.0000  11.0000
287   Default   13.000000           clean quiet   13.000000  10.0000  10.0000
1729  Default    8.000000          days arrival    8.000000   9.0000   9.0000
354   Default   36.000000       recommend place   36.000000   8.0000   8.0000
1002  Default   22.000000      easy communicate   22.000000   7.0000   7.0000
152   Default   10.000000            good place   10.000000   6.0000   6.0000
523   Default   12.000000           house clean   12.000000   5.0000   5.0000
212   Default   24.000000              san fran   24.000000   4.0000   4.0000
58    Default   26.000000           living room   26.000000   3.0000   3.0000
11    Default   76.000000      highly recommend   76.000000   2.0000   2.0000
26    Default   18.000000             felt like   18.000000   1.0000   1.0000
...       ...         ...                   ...         ...      ...      ...
3233  Topic12    1.533509       distance golden    3.877651   1.9520  -6.3050
1210  Topic12    1.534731          great airbnb    3.905094   1.9457  -6.3042
1750  Topic12    2.317884             good host    6.267549   1.8849  -5.8919
2579  Topic12    2.265803     place comfortable    6.319479   1.8540  -5.9147
1913  Topic12    2.291318        room beautiful    7.098852   1.7489  -5.9035
711   Topic12    2.336755           visiting sf    7.863229   1.6662  -5.8838
2048  Topic12    2.265755          easy parking    7.850090   1.6371  -5.9147
39    Topic12    7.530120      walking distance   78.310883   0.5379  -4.7137
122